In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
% matplotlib inline 
warnings.filterwarnings('ignore')

In [2]:
mapping = pd.read_csv("Mapping_csv.csv")
data = pd.read_csv("AnomalyReportData.csv")

In [3]:
data1 = data[data["Equipment Type"] == "MILK CHIILLING UNIT"]
data2 = data[data["Equipment Type"] == "MILK CHILLING UNIT"]
data = pd.concat([data1, data2])

In [4]:
data.shape

(4917, 84)

In [5]:
full_names = mapping["MD_id"].values
only_num = []
for i in full_names:
    
    user=i[i.find("y")+1:].split()[0]
    #print(user)
    only_num.append((user))
final_num = []
for i in only_num:
    l = i.replace("-","")
    #print(l)
    final_num.append(int(l))

In [6]:
mapping["num"] = final_num

In [7]:
index_list = []
check = data["Service Location"].values
want = mapping["num"].values
for i in range(len(check)):
    if check[i] not in want:
        index_list.append(i)

In [8]:
data.drop(data.index[index_list], inplace=True)
len(data.index)

760

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 51 to 50004
Data columns (total 84 columns):
Unnamed: 0                                   760 non-null int64
Request No                                   760 non-null int64
Request Status                               760 non-null object
Created Date                                 760 non-null object
SLA Crossed                                  760 non-null object
Priority                                     760 non-null object
Request Type                                 760 non-null object
Equipment Type                               760 non-null object
Equipment Sap Code                           760 non-null object
Equipment   Serial No                        760 non-null object
Equipment Description                        760 non-null object
Category                                     760 non-null object
SubCategory                                  760 non-null object
Major Issue                                  760 

In [10]:
colnames = data.columns.values
#keep will contain non null columns
keep = []
for i in colnames:
    if len(data[i].unique()) > 1:
        keep.append(i)
data = data[keep]

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 51 to 50004
Data columns (total 37 columns):
Unnamed: 0                       760 non-null int64
Request No                       760 non-null int64
Created Date                     760 non-null object
SLA Crossed                      760 non-null object
Equipment Sap Code               760 non-null object
Equipment Description            760 non-null object
Major Issue                      760 non-null object
Description                      757 non-null object
Escalation Level                 760 non-null object
Escalation - Internal Entity     368 non-null object
Sub Assigned                     8 non-null object
Assigned Date                    760 non-null object
Sub Assigned Date                8 non-null object
Escaltion Date                   368 non-null object
Due Date                         760 non-null object
Complete Date                    760 non-null object
Close Date                       760 non-null obje

In [12]:
st_streams = pd.HDFStore('streams.h5')

In [13]:
st_streams

<class 'pandas.io.pytables.HDFStore'>
File path: streams.h5
/streams/amb              frame        (shape->[1074241,68])
/streams/door             frame        (shape->[1074241,86])
/streams/mcu              frame        (shape->[1074241,86])
/streams/relay            frame        (shape->[1074241,86])
/streams/temp             frame        (shape->[1074241,86])

In [14]:
new_data = data[["Service Location" , "Created Date"]]

In [15]:
new_data.colmns = ["Booth no", "Created Date"]

In [16]:
new_data.isnull().any()

Service Location    False
Created Date        False
dtype: bool

In [17]:
power = st_streams['/streams/mcu']
temp = st_streams['/streams/temp']

In [18]:
power.columns.values

array(['mdb100', 'mdb101', 'mdb102', 'mdb103', 'mdb104', 'mdb105',
       'mdb106', 'mdb107', 'mdb108', 'mdb110', 'mdb112', 'mdb113',
       'mdb116', 'mdb117', 'mdb118', 'mdb119', 'mdb120', 'mdb121',
       'mdb122', 'mdb18', 'mdb20', 'mdb21', 'mdb23', 'mdb26', 'mdb27',
       'mdb28', 'mdb29', 'mdb30', 'mdb32', 'mdb34', 'mdb35', 'mdb36',
       'mdb37', 'mdb40', 'mdb41', 'mdb42', 'mdb43', 'mdb44', 'mdb45',
       'mdb46', 'mdb48', 'mdb49', 'mdb52', 'mdb53', 'mdb54', 'mdb55',
       'mdb56', 'mdb57', 'mdb58', 'mdb59', 'mdb60', 'mdb61', 'mdb63',
       'mdb64', 'mdb65', 'mdb66', 'mdb67', 'mdb68', 'mdb69', 'mdb70',
       'mdb71', 'mdb72', 'mdb73', 'mdb75', 'mdb76', 'mdb77', 'mdb78',
       'mdb79', 'mdb80', 'mdb81', 'mdb82', 'mdb83', 'mdb84', 'mdb85',
       'mdb87', 'mdb88', 'mdb89', 'mdb90', 'mdb91', 'mdb92', 'mdb93',
       'mdb94', 'mdb96', 'mdb97', 'mdb98', 'mdb99'], dtype=object)

In [19]:
print(new_data["Service Location"].unique())

[ 72  99  98 107  94 108 121  25  50  35   3  59  90 125  24  46  63 128
 104  65  74 127  95 123  92  84 109  78  86  16  73  28  26 120  12  44
  88 105  13  79 113  91  70  87 126  71   9  47  89  57  42  22  30  77
   7  10 122   2  38  29  83  53  33  81  97 110  76  45  60  64   1  43
  66 124  41  18 118  75   5  37 106  32  55  61  14   6  19  93  67  54
  52  85  96  80   4  20 103  69  56  27  62  49  39  68 119 117  21  40
 101  48 116  23  31   8 102  17  11  36  34  82]


In [20]:
check = []
full_names = power.columns.values
for i in full_names:
    user=i[i.find("b")+1:].split()[0]
    #print(user)
    check.append((int(user)))
    
in_service_not_in_power = []
for i in check:
    if i not in new_data["Service Location"].unique():
        in_service_not_in_power.append(i)

In [21]:
in_service_not_in_power

[100, 112, 58]

In [22]:
del power["mdb100"]
del power["mdb112"]
del power["mdb58"]
del temp["mdb100"]
del temp["mdb112"]
del temp["mdb58"]

In [23]:
not_in_service_in_power = []
for i in new_data["Service Location"].unique():
    if i not in check:
        not_in_service_in_power.append(i)

for i in not_in_service_in_power:
    new_data = new_data[new_data["Service Location"].values != i]

In [24]:
new_data.shape

(501, 2)

In [25]:
created_date = new_data["Created Date"].values
service_loc = new_data["Service Location"].values

In [26]:
new_created = []
for i in created_date:
    remove = i[:-7]
    new_created.append(remove)

In [27]:
l = pd.Series(data = power.index.values)
power.index = l.apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))

t = pd.Series(data = temp.index.values)
temp.index = t.apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))

In [28]:
int_index = list(range(power.shape[0]))
power["real_index"] = int_index

temp["real_index"] = int_index

In [29]:
for i in range(new_data.shape[0]):
    
    booth = "mdb"+ str(service_loc[i])
    something = power.ix[new_created[i]].real_index.values
    plotarray = power.ix[something[0]-5:something[0]+20][booth].values
    print(plotarray)
    print("*/"*40)
    plotarray = []

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/

In [31]:
for i in range(new_data.shape[0]):
    
    booth = "mdb"+ str(service_loc[i])
    something = temp.ix[new_created[i]].real_index.values
    plotarray = temp.ix[something[0]-5:something[0]+20][booth].values
    print(plotarray)
    print("*/"*40)
    plotarray = []

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/